# Convolutional Networks

### Load data

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub
import os
import PIL.Image as Image
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import sklearn as skl

os.environ["CUDA_VISIBLE_DEVICES"]= "-1" # Force Tensorflow on CPU instead of GPU (seems to avoid an error with my CUDA compatible GPU)

C:\Users\lyeso\anaconda3\envs\exts-ml\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Load training data form NPZ file

train_data = "train.npz"
X_tr = np.load(train_data)["features"]
y_tr = np.load(train_data)["labels"]
images_tr = np.load(train_data)["pixels"]
names_tr = np.load(train_data)["names"]

valid_data = "valid.npz"
X_val= np.load(valid_data)["features"]
y_val = np.load(valid_data)["labels"]
images_val = np.load(valid_data)["pixels"]
names_val = np.load(valid_data)["names"]

test_data = "test.npz"
X_te= np.load(test_data)["features"]
y_te = np.load(test_data)["labels"]
images_te = np.load(test_data)["pixels"]
names_te = np.load(test_data)["names"]

# Create a dictionnary for labels
labels_dict = {0: 'bike', 1 : 'car', 2: 'motorcycle', 3: 'other', 4:'truck', 5: 'van'}

In [3]:
images_tr[0].shape

(299, 299, 3)

### Let's use the train generator to have more images to train on and use in the Convolutional Network.

In [4]:
# Create image generator
train_generator = ImageDataGenerator(rescale=1/255, rotation_range = 10, horizontal_flip = True, vertical_flip = False, zoom_range = 0.2)
test_generator = ImageDataGenerator(rescale=1/255)

In [5]:
# Train, validation and test sets


# class_mode = categorical to have a one hot encoded "y" output.

trainset = train_generator.flow_from_directory(
    os.path.join('Course Project - SwissRoads', 'train'), batch_size =500,
    shuffle=True, color_mode='rgb', class_mode = 'categorical') 

validset = test_generator.flow_from_directory(
    os.path.join('Course Project - SwissRoads', 'valid'),batch_size =500,
    shuffle=False, color_mode='rgb', class_mode = 'categorical')
testset = test_generator.flow_from_directory(
    os.path.join('Course Project - SwissRoads', 'test'),batch_size =500,
    shuffle=False,color_mode='rgb', class_mode = 'categorical')

Found 280 images belonging to 6 classes.
Found 139 images belonging to 6 classes.
Found 50 images belonging to 6 classes.


### Let's create a 2 layers convolutional network

In [6]:
from tensorflow.keras.layers import Dropout

In [7]:
# Convolutional Network

model = keras.Sequential()


model.add(keras.layers.Conv2D(filters=128, kernel_size=5, strides=3, activation='relu', input_shape=(256, 256, 3)))

model.add(keras.layers.MaxPool2D(pool_size=2))


model.add(keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, activation='relu'))

model.add(keras.layers.MaxPool2D(pool_size=2))


model.add(keras.layers.Conv2D(filters=16, kernel_size=2, strides=1, activation='relu'))

model.add(keras.layers.MaxPool2D(pool_size=2))


model.add(keras.layers.Flatten())
model.add(Dropout(rate=0.3)) # because of overfitting

model.add(keras.layers.Dense(units=trainset.num_classes, activation='softmax'))


model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 84, 84, 128)       9728      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 42, 42, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 64)        73792     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 16)          4112      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 256)               0         
__________

In [8]:
# Compile the model
model.compile(optimizer=keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['acc'])

In [9]:
# End training when accuracy stops improving (optional)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=6)

In [10]:
trainset.num_classes

6

In [11]:
# Train model
history = model.fit_generator(
    generator = trainset, 
    validation_data = validset, 
    epochs=50, 
    callbacks=[early_stopping],
    verbose = 1,
    use_multiprocessing=False
)

Epoch 1/50
1/1 [==============================] - 10s 10s/step - loss: 1.8400 - acc: 0.1107 - val_loss: 1.7878 - val_acc: 0.1511
Epoch 2/50
1/1 [==============================] - 5s 5s/step - loss: 1.7871 - acc: 0.1679 - val_loss: 1.7835 - val_acc: 0.1799
Epoch 3/50
1/1 [==============================] - 7s 7s/step - loss: 1.7884 - acc: 0.1929 - val_loss: 1.7830 - val_acc: 0.1727
Epoch 4/50
1/1 [==============================] - 7s 7s/step - loss: 1.7824 - acc: 0.1964 - val_loss: 1.7772 - val_acc: 0.1799
Epoch 5/50
1/1 [==============================] - 8s 8s/step - loss: 1.7804 - acc: 0.1821 - val_loss: 1.7730 - val_acc: 0.1871
Epoch 6/50
1/1 [==============================] - 7s 7s/step - loss: 1.7818 - acc: 0.1786 - val_loss: 1.7695 - val_acc: 0.2446
Epoch 7/50
1/1 [==============================] - 7s 7s/step - loss: 1.7616 - acc: 0.2321 - val_loss: 1.7642 - val_acc: 0.2590
Epoch 8/50
1/1 [==============================] - 7s 7s/step - loss: 1.7630 - acc: 0.2643 - val_loss: 1.7617 

### By curiosity, let's try to use same model with numpy arrays and change y with One Hot Encoding to implement in the model

In [12]:
### Let's use a bigger dataset for training.

images = np.concatenate((images_tr,images_val), axis=0, out=None) 
y = np.concatenate((y_tr,y_val), axis=0, out=None) 

In [13]:
from sklearn.preprocessing import OneHotEncoder as ohe

nominal_transformer = ohe(handle_unknown='ignore', sparse = False)

y_ohe = nominal_transformer.fit_transform(y.reshape(-1,1))
print(y_ohe.shape)


y_tr_ohe = nominal_transformer.transform(y_tr.reshape(-1,1))
print(y_tr_ohe.shape)

y_val_ohe = nominal_transformer.transform(y_val.reshape(-1,1))
print(y_val_ohe.shape)

(419, 6)
(280, 6)
(139, 6)


In [20]:
model2 = keras.Sequential()



model2.add(keras.layers.Conv2D(filters=128, kernel_size=5, strides=3, activation='relu', input_shape=(299, 299, 3)))

model2.add(keras.layers.MaxPool2D(pool_size=2))


model2.add(keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, activation='relu'))

model2.add(keras.layers.MaxPool2D(pool_size=2))


model2.add(keras.layers.Conv2D(filters=16, kernel_size=2, strides=1, activation='relu'))

model2.add(keras.layers.MaxPool2D(pool_size=2))


model2.add(keras.layers.Flatten())
model2.add(Dropout(rate=0.3)) # because of overfitting

model2.add(keras.layers.Dense(units=trainset.num_classes, activation='softmax'))


model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 99, 99, 128)       9728      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 49, 49, 128)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 24, 24, 64)        73792     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 11, 11, 16)        4112      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 400)               0         
__________

In [21]:
# Compile the model
model2.compile(optimizer=keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['acc']) # loss = sparse_categorical_crossentropy does not need OHE

In [22]:


# Train model
history2 = model2.fit(
    x = images_tr, 
    y = y_tr_ohe, 
    epochs=50,
    validation_data = (images_val,y_val_ohe),
    callbacks=[early_stopping],
    verbose = 1,
    shuffle = True
)

Train on 280 samples, validate on 139 samples
Epoch 1/50
280/280 [==============================] - 6s 21ms/step - loss: 1.7748 - acc: 0.2214 - val_loss: 1.7414 - val_acc: 0.2374
Epoch 2/50
280/280 [==============================] - 5s 19ms/step - loss: 1.7539 - acc: 0.2536 - val_loss: 1.7366 - val_acc: 0.2374
Epoch 3/50
280/280 [==============================] - 5s 19ms/step - loss: 1.7367 - acc: 0.2714 - val_loss: 1.7132 - val_acc: 0.2590
Epoch 4/50
280/280 [==============================] - 6s 20ms/step - loss: 1.7146 - acc: 0.3536 - val_loss: 1.7020 - val_acc: 0.3381
Epoch 5/50
280/280 [==============================] - 6s 20ms/step - loss: 1.6804 - acc: 0.3429 - val_loss: 1.6916 - val_acc: 0.3309
Epoch 6/50
280/280 [==============================] - 6s 20ms/step - loss: 1.6819 - acc: 0.3714 - val_loss: 1.6568 - val_acc: 0.3525
Epoch 7/50
280/280 [==============================] - 6s 20ms/step - loss: 1.6281 - acc: 0.3857 - val_loss: 1.6108 - val_acc: 0.3813
Epoch 8/50
280/280 [===

Eventually, after several different architectures and trials, I always have a threshold of accuracy around 55% maximum. That is a lot worse than what we have with the high-level features.

Considering the low number of images my model is training on (~400) and the fact that it is not a binary prediction but we have 6 outputs possible. I consider 55% to be not that bad. 

Indeed high-level features from Inception v3 is trained on millions of images, has 24 million parameters and a depth of 159 !! (source : https://www.researchgate.net/figure/The-characteristics-of-deep-convnets-examined-in-this-study_tbl1_326421398)

### The lack of images to train on, and lack of depth and parameters, (for hardware reasons) make it complicated to reach the same kind of results as Google did. However I am glad my "simple ConvNet" is able to predict correctly more than half of times out of six outputs. 
**baseline would be way lower : ~17 % if the validation set had the same number of images for each output**



In [34]:
best_convnet_result = np.max(history2.history['val_acc']).round(3)
best_convnet_result

0.597

In [35]:
import csv

# Export Convnets results

data = ['ConvNets',best_convnet_result]


with open('Results.csv', 'a', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the data
    writer.writerow(data)